# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find a location for a new Italian restaurant in Barcelona, Spain. This project is targeted to people who want to open a Italian restaurant there.

We choose Barcelona because it is one of the most visited cities in the world by international visitors.

So, in order to find an optimal location in Barcelona we will try to find locations that are not already crowded with restaurants. We are interested in areas with no Italian restaurants near and also, we prefer locations close to city center.

We will use the different tools seen in the previous courses to find a few neighborhoods based on this criteria. 

## Data <a name="data"></a>

The factors that will influence our decission are:
* number of restaurants in each neighborhood 
* number of Italian restaurants and distance to them in each neighborhood
* distance of each neighborhood from city center

Following data sources will be needed to extract/generate the required information:
* Barcelona’s last published area of the neighborhoods of the city of Barcelona https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-superficie/resource/9cd3d989-03c2-4e39-b3a7-eec0eac53e9f
* Coordinates of the neighborhoods, we will use Nominatim to Geocode it.
* number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import random # library for random number generation
import numpy as np # library for vectorized computation


import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

For the Barcelona neighborhood data, we will extract the information from: https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-superficie/resource/9cd3d989-03c2-4e39-b3a7-eec0eac53e9f

In [11]:
d = pd.read_csv(".../2019_superficie.csv")
d=d.drop(['Any', 'Superfície (ha)'], axis=1)
d.head(10)


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc
5,2,Eixample,6,la Sagrada Família
6,2,Eixample,7,la Dreta de l'Eixample
7,2,Eixample,8,l'Antiga Esquerra de l'Eixample
8,2,Eixample,9,la Nova Esquerra de l'Eixample
9,2,Eixample,10,Sant Antoni


This data set doesn’t have the coordinates so we used Nominatim to Geocode the Neighbourhoods

In [12]:
geolocator=Nominatim(user_agent='bcn_geocoder')
lon=[]
lat=[]
for index, row in d.iterrows():
 location=geolocator.geocode(row['Nom_Barri']+',Barcelona')
 lon.append(location.longitude)
 lat.append(location.latitude)


In [13]:
d['Latitude']=lat
d['Longitude']=lon
d.head()

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Latitude,Longitude
0,1,Ciutat Vella,1,el Raval,41.379518,2.168368
1,1,Ciutat Vella,2,el Barri Gòtic,41.383395,2.176912
2,1,Ciutat Vella,3,la Barceloneta,41.380653,2.189927
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",41.388076,2.183398
4,2,Eixample,5,el Fort Pienc,41.395925,2.182325


In [14]:
d.shape

(73, 6)

Also, we create a map of Toronto to see the neighborhood

In [15]:
address = 'Barcelona, Spain'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of BArcelona are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of BArcelona are 41.3828939, 2.1774322.


In [16]:
map_bcn = folium.Map(location=[latitude, longitude], zoom_start=11
                    )

# add markers to map
for lat, lng, districte, neighborhood_bcn in zip(d['Latitude'], d['Longitude'], d['Nom_Districte'], d['Nom_Barri']):
    label = '{}, {}'.format(neighborhood_bcn, districte)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)  
    
map_bcn

Now let's define a query to search for Restaurants in Barcelona.

In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=...&client_secret=...&ll=41.3828939, 2.1774322&oauth_token=...&v=20180604&query=Restaurant&radius=2000&limit=100'


In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fc531444485170285a07d4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b62e49ef964a52096572ae3',
    'name': 'Bar Restaurant Cervantes',
    'location': {'address': 'C. Cervantes, 7',
     'lat': 41.381417829352245,
     'lng': 2.17755221585694,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.381417829352245,
       'lng': 2.17755221585694}],
     'distance': 164,
     'postalCode': '08004',
     'cc': 'ES',
     'city': 'Barcelona',
     'state': 'Cataluña',
     'country': 'España',
     'formattedAddress': ['C. Cervantes, 7',
      '08004 Barcelona Cataluña',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referr

In [19]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe_restaurant=json_normalize(venues)
dataframe_restaurant.head()

/home/leonpb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet,location.neighborhood
0,4b62e49ef964a52096572ae3,Bar Restaurant Cervantes,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1627149076,False,"C. Cervantes, 7",41.381418,2.177552,"[{'label': 'display', 'lat': 41.38141782935224...",164,08004,ES,Barcelona,Cataluña,España,"[C. Cervantes, 7, 08004 Barcelona Cataluña, Es...",NaN,NaN,NaN
1,543ec52a498e6e2ec8fd09d4,Habibi Restaurant,"[{'id': '52e81612bcbc57f1066b79ff', 'name': 'H...",v-1627149076,False,Rambla Sant Josep 75,41.381339,2.172969,"[{'label': 'display', 'lat': 41.38133855024618...",411,08002,ES,Barcelona,Cataluña,España,"[Rambla Sant Josep 75, 08002 Barcelona Cataluñ...",96607744,NaN,NaN
2,525476c511d2c5a744d4a95c,Arabia Bar & Restaurant,"[{'id': '4bf58dd8d48988d1c3941735', 'name': 'M...",v-1627149076,False,"C/ Escudellers, 25",41.379645,2.176328,"[{'label': 'display', 'lat': 41.379645, 'lng':...",373,08002,ES,Barcelona,Cataluña,España,"[C/ Escudellers, 25, 08002 Barcelona Cataluña,...",NaN,NaN,NaN
3,4bfd157bf61dc9b612719ede,ATN Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1627149076,False,"C. de la Canuda, 6",41.384568,2.172024,"[{'label': 'display', 'lat': 41.38456807218603...",488,08002,ES,Barcelona,Cataluña,España,"[C. de la Canuda, 6 (Pl. de la Vila de Madrid)...",80170178,Pl. de la Vila de Madrid,NaN
4,505f01d9e4b0351d974d1e83,Restaurant CentOnze,"[{'id': '4bf58dd8d48988d150941735', 'name': 'S...",v-1627149076,False,"La Rambla, 111",41.383796,2.171109,"[{'label': 'display', 'lat': 41.38379580826611...",537,NaN,ES,Barcelona,Cataluña,España,"[La Rambla, 111, Barcelona Cataluña, España]",NaN,NaN,NaN


In [20]:
dataframe_restaurant.shape

(50, 19)

Now let's define a query to search for Italian Restaurants in Barcelona.

In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=...&client_secret=...&ll=41.3828939, 2.1774322&oauth_token=...&v=20180604&query=Italian&radius=2000&limit=100'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fc53390893ce4051531fb3'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ca5a3967334236adcf51758',
    'name': 'Italian Job',
    'location': {'address': 'Enric Granados, 55',
     'crossStreet': 'Mallorca',
     'lat': 41.39055844141722,
     'lng': 2.1586963867238986,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.39055844141722,
       'lng': 2.1586963867238986}],
     'distance': 1782,
     'postalCode': '08007',
     'cc': 'ES',
     'city': 'Barcelona',
     'state': 'Cataluña',
     'country': 'España',
     'formattedAddress': ['Enric Granados, 55 (Mallorca)',
      '08007 Barcelona Cataluña',
      'España']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': T

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

dataframe_italian=json_normalize(venues)
dataframe_italian.head()

/home/leonpb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ca5a3967334236adcf51758,Italian Job,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1627149113,False,"Enric Granados, 55",Mallorca,41.390558,2.158696,"[{'label': 'display', 'lat': 41.39055844141722...",1782,08007,ES,Barcelona,Cataluña,España,"[Enric Granados, 55 (Mallorca), 08007 Barcelon...",NaN
1,4da9958c4df01c19b173f5e3,Dolce Vita Ly (puro Piacere Italiano),"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1627149113,False,"Calle Jaume I, 3",NaN,41.383036,2.177307,"[{'label': 'display', 'lat': 41.383036, 'lng':...",18,NaN,ES,Aguilar de Segarra,Cataluña,España,"[Calle Jaume I, 3, Barcelona Cataluña, España]",NaN
2,5bc2036d968f31002c9c8073,Italian Bistrot,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",v-1627149113,False,"Enric Granados, 115",NaN,41.392975,2.155333,"[{'label': 'display', 'lat': 41.392975, 'lng':...",2160,08008,ES,Barcelona,Cataluña,España,"[Enric Granados, 115, 08008 Barcelona Cataluña...",NaN
3,4b8c3d84f964a520f6c532e3,Rossini,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1627149113,False,"Plaça Reial, 13",NaN,41.379912,2.175864,"[{'label': 'display', 'lat': 41.37991239492408...",356,08002,ES,Barcelona,Cataluña,España,"[Plaça Reial, 13, 08002 Barcelona Cataluña, Es...",NaN
4,4c2617e5a852c928f5ece56c,Pagliotta Gelateria Italiana,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",v-1627149113,False,Carrer de Jaume I 15,NaN,41.383464,2.178055,"[{'label': 'display', 'lat': 41.38346441361018...",82,08002,ES,Barcelona,Cataluña,España,"[Carrer de Jaume I 15, 08002 Barcelona Cataluñ...",NaN


In [25]:
dataframe_italian.shape

(31, 18)